In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from vkoga_2L import VKOGA_2L
import tkernels
import kernels
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo

In [14]:
# Fetch dataset
parkinsons_telemonitoring = fetch_ucirepo(id=189)
X = parkinsons_telemonitoring.data.features
y = parkinsons_telemonitoring.data.targets

# Preprocess Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=None)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [15]:
model_matrix = VKOGA_2L(
    kernel=[kernels.Gaussian(), tkernels.Gaussian()], # quadratic Matern kernel used
    flag_2L_optimization=True,
    verbose=False,
    greedy_type='f_greedy',
    reg_par=0,
    restr_par=1e-2,
    tol_f=1e-10,
    tol_p=1e-10,
    reg_para_optim=1e-5,
    learning_rate=1e-3,
    n_epochs_optim=1000,
    batch_size=32,
    flag_optim_verbose=False
)

model_noMatrix = VKOGA_2L(
    kernel=[kernels.Gaussian(), tkernels.Gaussian()], # quadratic Matern kernel used
    flag_2L_optimization=False,
    verbose=False,
    greedy_type='f_greedy',
    reg_par=0,
    restr_par=1e-2,
    tol_f=1e-10,
    tol_p=1e-10,
    reg_para_optim=1e-5,
    learning_rate=1e-3,
    n_epochs_optim=1000,
    batch_size=32,
    flag_optim_verbose=False
)

If kernel optimization is desired, set flag_2L_optimization to True!


In [16]:
# Fit the model on training data
model_matrix.fit(X_train, y_train)
model_noMatrix.fit(X_train, y_train)

torch.Size([4700, 19])
torch.Size([4700, 2])
torch.Size([4700, 19])
torch.Size([4700, 2])


VKOGA_2L(flag_optim_verbose=False,
         kernel=<kernels.Gaussian object at 0x000002702634DCD0>,
         learning_rate=0.001, n_epochs_optim=1000, reg_para_optim=1e-05,
         restr_par=0.01, verbose=False)

In [17]:
# Predict on test data
predictions_matrix = model_matrix.predict(X_test)
predictions_noMatrix = model_noMatrix.predict(X_test)

In [18]:
# Print some predictions and ground truth
print("Predictions with 2 Layer Vkoga:\n", predictions_matrix[:5])
print("Ground truth:\n", y_test.numpy()[:5])

# predictions = predictions.flatten()
# Print Error
mse = np.mean((predictions_matrix - y_test.numpy()) ** 2)
print(f"MSE on Test Data: {mse:.6f}")

Predictions with 2 Layer Vkoga:
 [[-7.11278002e-002 -7.92192613e-002]
 [-1.64475167e-004 -1.83185847e-004]
 [ 1.37592271e-265  1.98649654e-265]
 [ 1.10087445e+000  1.14975663e+000]
 [-1.20767775e+000 -1.16533514e+000]]
Ground truth:
 [[ 0.10589432  0.18179175]
 [-1.0807815  -1.2147387 ]
 [-0.5973027  -0.19234233]
 [ 0.6192675   0.40236545]
 [-1.9387039  -2.0579658 ]]
MSE on Test Data: 0.334349


In [19]:
# Print some predictions and ground truth
print("Predictions without 2 Layer Vkoga:\n", predictions_noMatrix[:5])
print("Ground truth:\n", y_test.numpy()[:5])

# predictions = predictions.flatten()
# Print Error
mse = np.mean((predictions_noMatrix - y_test.numpy()) ** 2)
print(f"MSE on Test Data: {mse:.6f}")

Predictions without 2 Layer Vkoga:
 [[ 3.06219478e-02  4.02881466e-02]
 [ 5.94127225e-03  8.10010234e-03]
 [ 4.48025113e-04  5.48182790e-04]
 [-5.69559440e-03 -6.56154042e-03]
 [-1.76650783e+00 -1.89604640e+00]]
Ground truth:
 [[ 0.10589432  0.18179175]
 [-1.0807815  -1.2147387 ]
 [-0.5973027  -0.19234233]
 [ 0.6192675   0.40236545]
 [-1.9387039  -2.0579658 ]]
MSE on Test Data: 0.879638
